In [19]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.tools import ArxivQueryRun , WikipediaQueryRun , Tool
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities import ArxivAPIWrapper,  WikipediaAPIWrapper 
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [18]:
pdf_path = "qrcode_diffusion.pdf"
pdf_loader = PyPDFLoader(pdf_path)
pdf = pdf_loader.load()

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
chunker = SemanticChunker(embeddings=embedding_model , breakpoint_threshold_type="percentile" , min_chunk_size=100)
doc_splits = chunker.split_documents(pdf)

len(doc_splits)

db = Chroma.from_documents(doc_splits , embedding_model)
retriever = db.as_retriever()


In [15]:
wiki_tool = Tool(
    name="Wikipedia Search Tool",
    func=WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=5)).run,
    description="A tool for using wikipedia to get personalised results"
)

In [17]:
arxiv_tool = Tool(
    name="ARXIV search Tool",
    func = ArxivQueryRun(api_wrapper=ArxivAPIWrapper(top_k_results=5)).run,
    description="A tool for using arxiv to search through research papers"
)

In [20]:
pdf_tool = Tool(
    name= "QR CODE DIFFUSION RESEARCH PAPER TOOL",
    func = retriever.get_relevant_documents,
    description="A tool for using the knowledgebase of the pdf locally" 
    
)

In [21]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

tavily_tool = TavilySearchResults(max_results=5 , search_depth="advanced")


In [26]:
from langchain_google_genai.llms import GoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

llm = GoogleGenerativeAI(model="gemini-2.0-flash")

In [27]:
llm.invoke("hello")

'Hello! How can I help you today?\n'

In [28]:
tools = [ arxiv_tool , wiki_tool,  pdf_tool , tavily_tool ]

In [29]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose = True
)

C:\Users\hp\AppData\Local\Temp\ipykernel_8348\600513049.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [30]:
res = agent.run("Find ways to embed images in qr code")

C:\Users\hp\AppData\Local\Temp\ipykernel_8348\381962146.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = agent.run("Find ways to embed images in qr code")




> Entering new AgentExecutor chain...
I need to find information about embedding images in QR codes. I should start by searching for research papers on the topic to get a technical understanding.
Action: QR CODE DIFFUSION RESEARCH PAPER TOOL
Action Input: "embedding images in QR codes"
Observation: [Document(metadata={'page': 8, 'source': 'qrcode_diffusion.pdf'}, page_content='5\n[37] Open Source. Zxing: zebra crossing, 2013. 4\n[38] Hao Su, Jianwei Niu, Xuefeng Liu, Qingfeng Li, Ji Wan,\nand Mingliang Xu. Q-art code: Generating scanning-robust\nart-style qr codes by deformable convolution. In ACMMM ,\n2021. 3\n[39] Hao Su, Jianwei Niu, Xuefeng Liu, Qingfeng Li, Ji Wan,\nMingliang Xu, and Tao Ren. Artcoder: an end-to-end\nmethod for generating scanning-robust stylized qr codes. In\nCVPR , 2021. 2, 3\n9'), Document(metadata={'page': 8, 'source': 'qrcode_diffusion.pdf'}, page_content='5\n[37] Open Source. Zxing: zebra crossing, 2013. 4\n[38] Hao Su, Jianwei Niu, Xuefeng Liu, Qingfeng L

In [32]:
res


'Embedding images in QR codes can be achieved through various methods, including module deformation, module reshuffling, style transfer, and generative models like diffusion models. These methods aim to integrate reference images or styles into the QR code while maintaining scannability.'

'Embedding images in QR codes can be achieved through various methods, including module deformation, module reshuffling, style transfer, and generative models like diffusion models. These methods aim to integrate reference images or styles into the QR code while maintaining scannability.'